In [1]:
import sys
sys.path.append("../src")

import os

import pyvips 
import numpy as np
import pandas as pd 

from utils import (
    pad_img,
    get_args, 
    patchify,
    save_patches,
    save_patch_args, 
    get_target_shape,
    get_nearest_multiple
    )

/Users/ericmonzon/mambaforge/envs/histology/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = os.path.join("..", "..", "data", "images")
img_names = os.listdir(data_dir)

print("num images:", len(img_names))

num images: 127


In [3]:
img_paths = [os.path.join(data_dir, i) for i in img_names]

imgs = [pyvips.Image.new_from_file(f) for f in img_paths]

In [4]:
def get_target_shape(
    img_height: int, 
    img_width: int,
    img_patch_size: int,
    model_patch_size: int,
    model_window_size: int,
    ):

    """
    Calculates the target shape of an image so that its dimensions are multiples of the appropriate patch and window sizes.

    Parameters
    ----------
    img: np.ndarray
        The input image.

    img_patch_size: int
        The size of the patches to extract from the image.

    model_patch_size: int
        The patch size used in the model.

    model_window_size: int
        The window size used in the model.

    Returns
    -------
    target_size: Tuple[int, int]
        The adjusted (height, width) that are multiples of both the image patch size and the model's patch and window sizes.
    """

    # img_height, img_width = img.shape[0], img.shape[1]

    lcm_size = np.lcm(model_patch_size, model_window_size)
    lcm_size = lcm_size if lcm_size > 224 else 224

    def adjust_dimension(dim: int, patch_size: int) -> int:
        dim = get_nearest_multiple(dim, patch_size)
        num_patches = dim // patch_size
        num_patches = get_nearest_multiple(num_patches, lcm_size)
        return int(num_patches * patch_size)


    target_height = adjust_dimension(img_height, img_patch_size)
    target_width = adjust_dimension(img_width, img_patch_size)

    return target_height, target_width

In [5]:
data_dict = {
    "img": img_names,
    "height": [i.height for i in imgs],
    "width": [i.width for i in imgs]
}

df = pd.DataFrame(data_dict)

df.head()

,img,height,width
0,10336.ome.tif,23663,21566
1,10490.ome.tif,43328,33447
2,10548.ome.tif,40641,40072
3,10565.ome.tif,46012,40061
4,10576.ome.tif,19540,30407


In [6]:
img_patch_size = 224
model_patch_size = 4
model_window_size = 7

test = df.iloc[0]

In [7]:
target_height = []
target_width = []

for i in range(len(df)):
    row = df.iloc[i]
    height = row["height"]
    width = row["width"]

    target_shape = get_target_shape(height, width, img_patch_size, model_patch_size, model_window_size)
    target_height.append(target_shape[0])
    target_width.append(target_shape[1])


In [8]:
df["target_height"] = target_height
df["target_width"] = target_width

In [9]:
df.head()

,img,height,width,target_height,target_width
0,10336.ome.tif,23663,21566,50176,50176
1,10490.ome.tif,43328,33447,50176,50176
2,10548.ome.tif,40641,40072,50176,50176
3,10565.ome.tif,46012,40061,50176,50176
4,10576.ome.tif,19540,30407,50176,50176


In [10]:
df["target_width"].max()

np.int64(50176)

In [11]:
df["width"].max()

np.int64(49989)

In [12]:
df["height"].max()

np.int64(78287)

In [13]:
df["target_height"].max()

np.int64(100352)

In [14]:
df[df["height"] == df["height"].max()]

,img,height,width,target_height,target_width
114,15050.ome.tif,78287,46663,100352,50176


In [17]:
df["original_area"] = df["height"] * df["width"]
df["new_area"] = df["target_height"] * df["target_width"]

In [18]:
df["original_area"].sum()

np.int64(274495240835)

In [19]:
df["new_area"].sum()

np.int64(513596719104)

In [ ]:
274_495_240_835
513_596_719_104